## Importing Libraries and Data

In [1]:
!pip install pandas
!pip install tensorflow
!pip install tensorflow_hub
!pip install tokenization
!pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 67 kB 3.0 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import re
from bert import tokenization
import string
tf.gfile = tf.io.gfile

import os

os.getcwd()

'/content'

In [3]:
from google.colab import drive
drive.mount('/content/drive')


train = pd.read_csv("/content/drive/MyDrive/python/train.csv")
test = pd.read_csv("/content/drive/MyDrive/python/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/python/sample_submission.csv")

Mounted at /content/drive


## Helper functions

In [4]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:

        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [5]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    dense_layer1 = Dense(units=256, activation='relu')(clf_output)
    dense_layer1 = Dropout(0.5)(dense_layer1)
    out = Dense(1, activation='sigmoid')(dense_layer1)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Data cleaning

In [6]:
def lowercase_text(text):
    return text.lower()

train.text=train.text.apply(lambda x: lowercase_text(x))
test.text=test.text.apply(lambda x: lowercase_text(x))

In [7]:
def remove_noise(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

train.text=train.text.apply(lambda x: remove_noise(x))
test.text=test.text.apply(lambda x: remove_noise(x))

In [8]:
train.text.head(5)

0    our deeds are the reason of this earthquake ma...
1                forest fire near la ronge sask canada
2    all residents asked to shelter in place are be...
3     people receive wildfires evacuation orders in...
4    just got sent this photo from ruby alaska as s...
Name: text, dtype: object

## Pre-Training (BERT)

In [9]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [10]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [11]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

## Modelling

In [12]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [13]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=8,
    callbacks=[checkpoint],
    batch_size=10
)

Epoch 1/8
609/609 [==============================] - 1302s 2s/step - loss: 0.4517 - accuracy: 0.8013 - val_loss: 0.3979 - val_accuracy: 0.8162
Epoch 2/8
609/609 [==============================] - 1260s 2s/step - loss: 0.2904 - accuracy: 0.8905 - val_loss: 0.3878 - val_accuracy: 0.8470
Epoch 3/8
609/609 [==============================] - 1259s 2s/step - loss: 0.1603 - accuracy: 0.9450 - val_loss: 0.5393 - val_accuracy: 0.8280
Epoch 4/8
609/609 [==============================] - 1277s 2s/step - loss: 0.0883 - accuracy: 0.9672 - val_loss: 0.7390 - val_accuracy: 0.8011
Epoch 5/8
609/609 [==============================] - 1279s 2s/step - loss: 0.0840 - accuracy: 0.9709 - val_loss: 0.6443 - val_accuracy: 0.8207
Epoch 6/8
609/609 [==============================] - 1279s 2s/step - loss: 0.0745 - accuracy: 0.9690 - val_loss: 0.7701 - val_accuracy: 0.8109
Epoch 7/8
609/609 [==============================] - 1264s 2s/step - loss: 0.0656 - accuracy: 0.9742 - val_loss: 0.7941 - val_accuracy: 0.8188

In [14]:
metrics=pd.DataFrame(model.history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,0.451718,0.801314,0.397911,0.816152
1,0.290415,0.890476,0.387810,0.847012
2,0.160322,0.944992,0.539266,0.827971
3,0.088295,0.967159,0.739038,0.801051
4,0.084006,0.970936,0.644346,0.820749
5,0.074538,0.968966,0.770099,0.810900
6,0.065631,0.974220,0.794148,0.818779
7,0.057097,0.975862,0.740897,0.818779


In [15]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [16]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)